# KNN(K-최근접 이웃) 분류

- 기술적으로 예측을 만들기 위해 모델을 훈련하지 않고 가장 가까운 k개의 샘플에서 다수의 클래스를 그 샘플의 클래스로 예측
- 분류(classification)알고리즘 으로 유사한 특성을 가진 데이터는 유사한 범주에 속하는 경행이 있다고 가정
- KNN사용시 모든 특성을 고르게 반영하기 위해 정규화 진행
    * 최소값을 0, 최대값을 1로 고정한 뒤 모든 값을 0과 1사이 값으로 변환
    * 평균과 표준편차를 활용해서 평균으로부터 얼마나 떨어져 있는지 z-점수로 변환
- K 개수 선택- 모든 값을 실제로 테스트 하면서 분류 정확도를 계산 하는 과정에서 발견가능
- KNN은 주변 다른 이웃까지 충분히 고려하지 않았을 때 오버피팅 발생
- k가 너무 큰경우 underfiting(과소적합)발생, 너무 작은경우 overfiting발생


### 샘플의 최근접 이웃찾기

In [1]:
#샘플의 최근접 이웃찾기

from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris() # 데이터 로드
features = iris.data
standardizer = StandardScaler() # 표준화 객체 생성
features_standardized = standardizer.fit_transform(features) # 특성을 표준화

# k=2인 최근접 이웃 모델 생성
nearest_neighbors = NearestNeighbors(n_neighbors=2).fit(features_standardized)
new_observation = [ 1, 1, 1, 1] #New Sample Data

# New 샘플과 가장 가까운 이웃의 인덱스와 거리 탐색
distances, indices = nearest_neighbors.kneighbors([new_observation])
features_standardized[indices] # 최근접 이웃을 확인

nearestneighbors_euclidean = NearestNeighbors( n_neighbors=2, metric='euclidean').fit(features_standardized)
distances # 거리 확인

# 유클리디안 거리를 기반으로 각 샘플에 대해 (자기 자신을 포함한) 세 개의 최근접 이웃 탐색
nearestneighbors_euclidean = NearestNeighbors(n_neighbors=3, metric="euclidean").fit(features_standardized) 

In [2]:
# 각 샘플의 (자기 자신을 포함한) 3개의 최근접 이웃을 나타내는 리스트의 리스트
nearest_neighbors_with_self = nearestneighbors_euclidean.kneighbors_graph( features_standardized).toarray()
# 최근접 이웃 중에서 1로 표시된 자기 자신 제외
for i, x in enumerate(nearest_neighbors_with_self ):
    x[i] = 0

# 첫 번째 샘플에 대한 두 개의 최근접 이웃 확인
nearest_neighbors_with_self[0]

# 이 샘플과 가장 가까운 이웃의 다섯개의 인덱스 탐색
indices = nearest_neighbors.kneighbors([new_observation], n_neighbors=5, return_distance=False)
features_standardized[indices] # 최근접 이웃을 확인

# 반경 0.5 안에 있는 모든 샘플의 인덱스 탐색
indices = nearest_neighbors.radius_neighbors( [new_observation], radius=0.5, return_distance=False)
features_standardized[indices[0]] # 반경 내의 이웃 확인

# 반경 내의 이웃을 나타내는 리스트의 리스트
nearest_neighbors_with_self = nearest_neighbors.radius_neighbors_graph( [new_observation], radius=0.5).toarray()
nearest_neighbors_with_self[0] # 첫 번째 샘플에 대한 반경 내의 이웃 확인

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### k-최근접 이웃 분류기

In [3]:
# k-최근접 이웃 분류기
#클래스를 모르는 샘플이 주어졌을 때 이웃한 샘플 클래스 기반으로 샘플 클래스 예측

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

iris = datasets.load_iris() # 데이터 로드
X = iris.data
y = iris.target

standardizer = StandardScaler() # 표준화 객체
X_std = standardizer.fit_transform(X) # 특성을 표준화

# 5개의 이웃을 사용한 KNN 분류기 훈련
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1).fit(X_std, y)
new_observations = [[ 0.75, 0.75, 0.75, 0.75],
    [ 1, 1, 1, 1]] # 두 개의 샘플을 만듬

knn.predict(new_observations) # 두 샘플의 클래스를 예측
knn.predict_proba(new_observations) # 각 샘플이 세 클래스에 속할 확률을 확인

array([[0. , 0.6, 0.4],
       [0. , 0. , 1. ]])

In [4]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets

boston = datasets.load_boston() # 데이터 로드
features = boston.data[:,0:2] #두 개의 특성만 선택
target = boston.target

# 최근접 회귀 모델 객체 생성
knn_regressor = KNeighborsRegressor(n_neighbors=10)
model = knn_regressor.fit(features, target) # 모델 훈련
# 첫 번째 샘플의 타깃 값을 예측하고 1000을 곱함
model.predict(features[0:1])[0]*1000

import numpy as np

indices = model.kneighbors(features[0:1], return_distance=False)
np.mean(target[indices]) * 1000

32440.000000000004

### 최선의 이웃 개수 결정

- KNN 분류기에 각기 다른 k값으로 5-폴드 교차검증을 수행하는 GridSerarchCV사용

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris() # 데이터 로드
features = iris.data
target = iris.target

standardizer = StandardScaler() # 표준화 객체 생성
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1) # KNN 분류기 객체 생성
pipe = Pipeline([("standardizer", standardizer), ("knn", knn)]) # 파이프라인 생성

# 탐색 영역의 후보를 만듭니다.
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

# 그리드 서치 객체 생성
classifier = GridSearchCV( pipe, search_space, cv=5, verbose=0).fit(features, target)

# 최선의 이웃 개수 (k)
classifier.best_estimator_.get_params()["knn__n_neighbors"]

6

### 반지름 기반의 최근접 이웃 분류기(RadiusNeighborsClassifier)

- 샘플의 클래스가 주어진 반지름 r 이내에 있는 모든 샘플의 클래스로 부터 예측
- radius 매개변수로 고정 영역의 반지름을 지정하여 이웃 샘플 결정
- outlier_lavel 매개변수는 반지름 내에 다른 샘플이 하나도 없는 샘플에 부여할 레이블 지정

In [6]:
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
i
ris = datasets.load_iris() # 데이터 로드
features = iris.data
target = iris.target
standardizer = StandardScaler() # 표준화 객체 생성
features_standardized = standardizer.fit_transform(features) # 특성을 표준화

# 반지름 이웃 분류기를 훈련합니다.
rnn = RadiusNeighborsClassifier( radius=.5, n_jobs=-1).fit(features_standardized, target)
new_observations = [[ 1, 1, 1, 1]] # 두 개의 샘플을 만듭니다.
rnn.predict(new_observations) # 두 샘플의 클래스를 예측

# 반지름 이웃 분류기를 훈련합니다.
rnn = RadiusNeighborsClassifier( radius=.5, outlier_label=-1, n_jobs=-1).fit(features_standardized, target)
rnn.predict([[100, 100, 100, 100]])

C:\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:566: UserWarning: Outlier label -1 is not in training classes. All class probabilities of outliers will be assigned with 0.
  ''.format(self.outlier_label_[k]))


array([-1])